In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

from transformer import Transformer, TOK_LEN, PAD_TOK, MAX_INST_LEN
from stacksyms import checkLabels, generateDebugLabel, getMaxFrameSize, getFunctions, parseELF

file2funcs = dict() # = parseDirectory(debugFiledir)

#debugFilepath = 'data/cross-compile-dataset/bin/static/gcc/og/parallel'
debugFiledir = 'data/cross-compile-dataset/bin/static/gcc/og'
#debugFiledir = '.'
#debugFilepath = 'data/cross-compile-dataset/bin/static/gcc/og/sum'
#debugFilepath = 'data/spec2006/bzip2'

In [2]:
# TODO get frame size from .eh_frame section and compare func.variables against GDB's 'info scope func.name' output
for fname in next(os.walk(debugFiledir))[2]:
    debugFilepath = debugFiledir + os.sep + fname
    with open(debugFilepath, 'rb') as fstream:
        if fstream.peek(1)[:4].hex() == b'\x7fELF'.hex():
            functions = parseELF(debugFilepath, validateWithGDB=False)
            checkLabels(functions)

02/15/2021 16:38:53 INFO:parseELF:Trying to parse data/cross-compile-dataset/bin/static/gcc/og/sum as ELF
02/15/2021 16:38:53 INFO:parseDWARF:File has debug info..
02/15/2021 16:38:54 INFO:parseELF:Found 87 functions.
02/15/2021 16:38:54 INFO:collectFrameInfo:has .eh_frames
02/15/2021 16:38:54 INFO:collectFrameInfo:Found frame info for function sysv_sum_file.
02/15/2021 16:38:54 INFO:collectFrameInfo:Found frame info for function bsd_sum_file.
02/15/2021 16:38:54 INFO:collectFrameInfo:Found frame info for function usage.
02/15/2021 16:38:54 INFO:collectFrameInfo:Found frame info for function main.
02/15/2021 16:38:54 INFO:collectFrameInfo:Found frame info for function close_stdout_set_file_name.
02/15/2021 16:38:54 INFO:collectFrameInfo:Found frame info for function close_stdout_set_ignore_EPIPE.
02/15/2021 16:38:54 INFO:collectFrameInfo:Found frame info for function close_stdout.
02/15/2021 16:38:54 INFO:collectFrameInfo:Found frame info for function fdadvise.
02/15/2021 16:38:54 INFO

02/15/2021 16:38:55 INFO:generateDebugLabel:Function main has 6 stack elements out of 12 total.
02/15/2021 16:38:55 INFO:generateDebugLabel:Function sysv_sum_file has 11 stack elements out of 21 total.
02/15/2021 16:38:55 INFO:generateDebugLabel:Function bsd_sum_file has 10 stack elements out of 16 total.
02/15/2021 16:38:55 INFO:generateDebugLabel:Function usage has 4 stack elements out of 5 total.
02/15/2021 16:38:55 INFO:generateDebugLabel:Function close_stdout has 3 stack elements out of 4 total.
02/15/2021 16:38:55 INFO:generateDebugLabel:Function close_stdout_set_ignore_EPIPE has 1 stack elements out of 2 total.
02/15/2021 16:38:55 INFO:generateDebugLabel:Function close_stdout_set_file_name has 1 stack elements out of 2 total.
02/15/2021 16:38:55 INFO:generateDebugLabel:Function fadvise has 2 stack elements out of 4 total.
02/15/2021 16:38:55 INFO:generateDebugLabel:Function fdadvise has 1 stack elements out of 5 total.
02/15/2021 16:38:55 INFO:generateDebugLabel:Function human_o

02/15/2021 16:38:55 INFO:generateDebugLabel:Function argmatch_invalid has 4 stack elements out of 8 total.
02/15/2021 16:38:55 INFO:generateDebugLabel:Function argmatch has 10 stack elements out of 15 total.
02/15/2021 16:38:55 INFO:generateDebugLabel:Function c_strcasecmp has 1 stack elements out of 7 total.
02/15/2021 16:38:55 INFO:generateDebugLabel:Function close_stream has 7 stack elements out of 8 total.
02/15/2021 16:38:55 WARNING:checkLabels:Function close_stream has VOID or VARIADIC type stack objects; cannot determine byte size!
02/15/2021 16:38:55 INFO:generateDebugLabel:Function locale_charset has 4 stack elements out of 6 total.
02/15/2021 16:38:55 INFO:generateDebugLabel:Function get_charset_aliases has 12 stack elements out of 22 total.
02/15/2021 16:38:55 INFO:parseELF:Trying to parse data/cross-compile-dataset/bin/static/gcc/og/nohup as ELF
02/15/2021 16:38:55 INFO:parseDWARF:File has debug info..


main (48 by offset / 48 by size) => [8, 8, 8, 8, 8, 8]
sysv_sum_file (8928 by offset / 8905 by size) => [652, 8192, 8, 8, 8, 8, 8, 8, 8, 1, 4]
bsd_sum_file (736 by offset / 713 by size) => [652, 8, 8, 8, 8, 8, 8, 8, 1, 4]
usage (32 by offset / 32 by size) => [8, 8, 8, 8]
close_stdout (24 by offset / 24 by size) => [8, 8, 8]
close_stdout_set_ignore_EPIPE (8 by offset / 8 by size) => [8]
close_stdout_set_file_name (8 by offset / 8 by size) => [8]
fadvise (16 by offset / 16 by size) => [8, 8]
fdadvise (8 by offset / 8 by size) => [8]
human_options (16 by offset / 16 by size) => [8, 8]
humblock (64 by offset / 48 by size) => [8, 8, 8, 8, 8, 8]
default_block_size (8 by offset / 8 by size) => [8]
human_readable (136 by offset / 208 by size) => [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 16, 8, 8, 8, 8, 4, 4, 8, 8, 8, 8, 8, 8, 4, 4, 8, 8]
group_number (128 by offset / 97 by size) => [41, 8, 8, 8, 8, 8, 8, 8]
adjust_value (8 by offset / 24 by size) => [8, 16]
set_program_name (16 by offset / 16 by size) =>

02/15/2021 16:38:55 INFO:parseELF:Found 78 functions.
02/15/2021 16:38:55 INFO:collectFrameInfo:has .eh_frames
02/15/2021 16:38:56 INFO:collectFrameInfo:Found frame info for function usage.
02/15/2021 16:38:56 INFO:collectFrameInfo:Found frame info for function main.
02/15/2021 16:38:56 INFO:collectFrameInfo:Found frame info for function set_cloexec_flag.
02/15/2021 16:38:56 INFO:collectFrameInfo:Found frame info for function dup_cloexec.
02/15/2021 16:38:56 INFO:collectFrameInfo:Found frame info for function close_stdout_set_file_name.
02/15/2021 16:38:56 INFO:collectFrameInfo:Found frame info for function close_stdout_set_ignore_EPIPE.
02/15/2021 16:38:56 INFO:collectFrameInfo:Found frame info for function close_stdout.
02/15/2021 16:38:56 INFO:collectFrameInfo:Found frame info for function fd_reopen.
02/15/2021 16:38:56 INFO:collectFrameInfo:Found frame info for function file_name_concat.
02/15/2021 16:38:56 INFO:collectFrameInfo:Found frame info for function longest_relative_suffix

...

02/15/2021 16:42:14 INFO:collectFrameInfo:Found frame info for function close_stream.
02/15/2021 16:42:14 INFO:collectFrameInfo:Found frame info for function open_safer.
02/15/2021 16:42:14 INFO:collectFrameInfo:Found frame info for function hash_pjw.
02/15/2021 16:42:14 INFO:collectFrameInfo:Found frame info for function get_charset_aliases.
02/15/2021 16:42:14 INFO:collectFrameInfo:Found frame info for function locale_charset.
02/15/2021 16:42:14 INFO:collectFrameInfo:Found frame info for function mb_width_aux.
02/15/2021 16:42:14 INFO:collectFrameInfo:Found frame info for function mb_copy.
02/15/2021 16:42:14 INFO:collectFrameInfo:Found frame info for function is_basic.
02/15/2021 16:42:14 INFO:collectFrameInfo:Found frame info for function knuth_morris_pratt.
02/15/2021 16:42:14 INFO:collectFrameInfo:Found frame info for function knuth_morris_pratt_multibyte.
02/15/2021 16:42:14 INFO:collectFrameInfo:Found frame info for function mbsstr.
02/15/2021 16:42:14 INFO:collectFrameInfo:Fo

02/15/2021 16:42:15 INFO:generateDebugLabel:Function hash_free has 4 stack elements out of 8 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function hash_clear has 4 stack elements out of 8 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function hash_initialize has 7 stack elements out of 13 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function compute_bucket_size has 1 stack elements out of 4 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function check_tuning has 2 stack elements out of 4 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function raw_comparator has 1 stack elements out of 3 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function raw_hasher has 1 stack elements out of 3 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function hash_reset_tuning has 1 stack elements out of 2 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function next_prime has 2 stack elements out of 3 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function is_prime ha

02/15/2021 16:42:15 INFO:generateDebugLabel:Function set_char_quoting has 2 stack elements out of 8 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function set_quoting_style has 1 stack elements out of 3 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function get_quoting_style has 1 stack elements out of 2 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function clone_quoting_options has 4 stack elements out of 7 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function strcaseeq6 has 3 stack elements out of 6 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function same_name has 10 stack elements out of 19 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function free_cwd has 2 stack elements out of 3 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function restore_cwd has 1 stack elements out of 2 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function save_cwd has 2 stack elements out of 3 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function strnlen1 

02/15/2021 16:42:15 INFO:generateDebugLabel:Function freea has 3 stack elements out of 6 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function mmalloca has 2 stack elements out of 6 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function mbslen has 5 stack elements out of 6 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function dup_safer has 1 stack elements out of 2 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function printf_fetchargs has 1 stack elements out of 6 total.
02/15/2021 16:42:15 INFO:generateDebugLabel:Function printf_parse has 25 stack elements out of 40 total.


main (88 by offset / 61 by size) => [1, 4, 8, 8, 8, 8, 8, 8, 8]
usage (32 by offset / 32 by size) => [8, 8, 8, 8]
add_excluded_fs_type (16 by offset / 24 by size) => [8, 8, 8]
add_fs_type (16 by offset / 24 by size) => [8, 8, 8]
get_all_entries (16 by offset / 16 by size) => [8, 8]
get_entry (24 by offset / 24 by size) => [8, 8, 8]
get_point (232 by offset / 216 by size) => [8, 144, 8, 8, 8, 8, 8, 8, 8, 8]
get_disk (259 by offset / 242 by size) => [1, 1, 8, 8, 8, 8, 8, 144, 8, 8, 8, 8, 8, 8, 8]
last_device_for_mount (32 by offset / 32 by size) => [8, 8, 8, 8]
get_dev (1112 by offset / 1095 by size) => [8, 8, 8, 8, 8, 56, 64, 64, 144, 653, 8, 8, 8, 8, 8, 8, 8, 1, 8, 8, 1]
add_to_grand_total (32 by offset / 32 by size) => [8, 8, 8, 8]
get_field_values (48 by offset / 48 by size) => [8, 8, 8, 8, 8, 8]
has_uuid_suffix (8 by offset / 8 by size) => [8]
add_uint_with_neg_flag (8 by offset / 8 by size) => [8]
df_readable (56 by offset / 64 by size) => [8, 8, 8, 8, 8, 8, 8, 8]
known_value (8 by

In [ ]:
functions = parseELF(debugFilepath)

In [ ]:
import logging; logging.getLogger().setLevel(logging.INFO)
for func in functions:
    frame = getMaxFrameSize(func)
    label = generateDebugLabel(func)
    print(f"{func.name} ({frame} by offset) => {label}")

In [ ]:
from stacksyms import locExprHasOffset


def getMinOffLoc(stkElm):
    return min(loc.expr[1] for loc in stkElm.locations)

def getStackLocation(stkElm):
    return [loc.expr for loc in filter(locExprHasOffset, stkElm.locations)]

def getStackElements(func):
    potentialStkElms = func.parameters + func.variables
    stkElms = [stkElm for stkElm in potentialStkElms if any(filter(locExprHasOffset, stkElm.locations))]
    return sorted(stkElms, key=getMinOffLoc)

def generateDebugLabel(stackElements):
    #byOffset = sorted(stackElements, key=lambda stkElm :
    return list(map(getStackLocation, stackElements))

for func in functions:
    stkElms = getStackElements(func)
    if len(stkElms) != lenPotStkElms:
        print(f"{len(stkElms)} != {lenPotStkElms} for {stkElms}")
    #labels  = generateDebugLabel(stkElms)
    #print(func.name+"\n", "\n".join([f"{stkElm.name} => {str(label)}" for stkElm, label in zip(stkElms, labels)]))
    print(f"{func.name} => [{', '.join(stkElm.name+'@ebp%+d'%getMinOffLoc(stkElm) for stkElm in stkElms)}]")

In [ ]:
types = set()
count = 0
for func in functions:
    print(func.name)
    for stkElm in func.variables:
        if not stkElm.type.is_qualified_type and stkElm.type.byte_size is None:
            print(stkElm.type, stkElm.type.byte_size, stkElm.type.array_count, stkElm.type.element, stkElm.type.is)
        types |= {stkElm.type}
        count += 1
    print()
    #frameSize = getMaxFrameSize(func)
    #paramDescrs = [f'{var} (bytesize = {var.type.array_count})' for var in func.parameters]
    #localDescrs = [f'{var} (bytesize = {var.type.array_count})' for var in func.variables]
    #print(f'{func.name} max frame size is {frameSize} bytes, params are {paramDescrs} locals are {localDescrs}')
    #print()
print(count, len(types))

In [ ]:
quotearg_n_style_mem = next((f for f in functions if f.name == 'quotearg_n_style_mem'), None)
print(quotearg_n_style_mem.frame)

In [ ]:
for fname in next(os.walk(debugFiledir))[2]:
    debugFilepath = debugFiledir + os.sep + fname
    with open(debugFilepath, 'rb') as fstream:
        if not fstream.peek(1)[:4].hex() == b'\x7fELF'.hex():
            continue
        print('Is ELF, checking for debug info..')
        elf = ELFFile(fstream)
        print(elf.get_machine_arch())

In [ ]:
from elftools.elf.elffile import ELFFile
from elftools.dwarf.constants import DW_LNS_copy, DW_LNS_set_file, DW_LNE_define_file, \
                                     DW_LNS_set_basic_block, DW_LNS_set_prologue_end, DW_LNS_set_isa
from dwarf_import.model.module import Module
from dwarf_import.io.dwarf_import import DWARFDB, DWARFImporter, place_component_in_module_tree

for fname in next(os.walk(debugFiledir))[2]:
    debugFilepath = debugFiledir + os.sep + fname
    print(f'Opening {debugFilepath} to check magic..')
    with open(debugFilepath, 'rb') as fstream:
        if not fstream.peek(1)[:4].hex() == b'\x7fELF'.hex():
            continue
        print('Is ELF, checking for debug info..')
        elf = ELFFile(fstream)
        if not elf.has_dwarf_info():
            print('does not contain DWARF info')
            continue
        print('Has debug info, parsing functions..')
        dwarfInfo = elf.get_dwarf_info()
        module, importer = parseDWARF(elf)
        file2funcs[debugFilepath] = getFunctions(module)
        #handleLineprogram(dwarfInfo)
        collectFrameInfo(file2funcs[debugFilepath], elf)


In [ ]:
#from elftools.dwarf.descriptions import describe_reg_name

'''should probably be named getMaxFrameSize'''
def getFrameSize(function):
    # The number we get here statically from the .eh_frame section can actually be validated using GDB:
    # ./gdb path/to/prog
    # (gdb) set confirmation off
    # (gdb) break {func.name}
    # (gdb) r
    # (gdb) rbreak .
    # (gdb) c
    # (gdb) info frame
    #  at this point "frame at 0xADDRESS_A" - "called by frame at 0xADDRESS_B" should match our number below
    funcFrameRegs = [(key, val) for d in function.frame for key, val in d.items() if type(key)==int]
    return abs(min(funcFrameRegs, key=lambda t : t[1].arg)[1].arg)

sumFuncs = file2funcs['data/cross-compile-dataset/bin/static/gcc/og/sum']
sumMain = list(filter(lambda func : func.name == 'main', sumFuncs))[0]
print([f'{func.name} => {getFrameSize(func)}' for func in sumFuncs])
#print([describe_reg_name(regno, 'x64') + " => " + str(rule.arg) for (regno, rule) in sumMainFrameRegs])



In [ ]:
#file2funcs = parseDirectory(debugFiledir)

#functions = parseELF(debugFilepath)
#generateFeatures(functions)
#assert(not any(filter(lambda x : PAD_TOK in x, data)))

#main = list(filter(lambda x : x.name=='main', functions))[0]
#main_x = generateFeatures(main)


In [ ]:
# TODO: newCollectLocals(gdbOut, scopeQueries, functions) and newCollectDisas(gdbOut, disasQueries, functions)
#for scope, func in zip(scopeResults, functions):
#    collectLocals(scope, functions[func])
for disas, func in zip(gdbOut, functions):
    func.disas = [tuple(line.strip().split('\\t')) for line in disas[1:-1]]

In [ ]:
X, Y, Z = [], [], []
for func in functions:
    print('////////////////////////')
    print(func.name, hex(func.start), func.frame_base)
    for lvar in func.variables:
        print(lvar.name, lvar.type, "(bytesize = %d)"%lvar.type.byte_size)
        for loc in lvar.locations:
            print(hex(loc.begin) + " to " + hex(loc.end) + ": " \
            + str(loc.type)[13:]
            + str(loc.expr))
        print('')
#    X += [func] #generateFeature(func, functions)]
#    Y += [generateLabel(func, functions)]
#    Z += [generateDebugLabel(func, functions)]
#print([x+" => "+ str(y) for x,y in zip(X,Z)])

#func = 'quotearg_n_style_colon'

In [ ]:
'''Doesnt appear to be super useful for stack symbolization as compilers dont seem to emit relevant information'''
def handleLineprogram(dwarfInfo):
    print('looping through compilation units..')
    for cu in dwarfInfo.iter_CUs():
        lp = dwarfInfo.line_program_for_CU(cu)
        if lp == None:
            print('DWARF info is missing a line program for this CU')
            print(cu.keys())
            continue
        cu_filename = lp['file_entry'][0].name.decode('latin-1')
        if len(lp['include_directory']) > 0:
            dir_index = lp['file_entry'][0].dir_index
            if dir_index > 0:
                idir = lp['include_directory'][dir_index - 1]
            else:
                idir = b'.'
            cu_filename = '%s/%s' % (idir.decode('latin-1'), cu_filename)
        print(f'CU: {cu_filename}')
        #print(f'File name                            Line number    Starting address')
        for entry in lp.get_entries():
            '''
            if entry.state is None:
                # Special handling for commands that don't set a new state
                if entry.command == DW_LNS_set_file:
                    file_entry = lp['file_entry'][entry.args[0] - 1]
                    if file_entry.dir_index == 0:
                        # current directory
                        print('\n./%s:[++]' % (
                            file_entry.name.decode('latin-1')))
                    else:
                        print('\n%s/%s:' % (
                            lp['include_directory'][file_entry.dir_index - 1].decode('latin-1'),
                            file_entry.name.decode('latin-1')))
                elif entry.command == DW_LNE_define_file:
                    print('%s:' % (
                        lp['include_directory'][entry.args[0].dir_index].decode('latin-1')))
                elif entry.command in [DW_LNS_set_basic_block, DW_LNS_set_prologue_end, DW_LNS_set_isa]:
                    print('=============================')
                    print(repr(entry))
                    print('=============================')
                else:
                    print('=============================')
                    print('OTHER COMMAND: ' + str(entry))
                    print('=============================')
            elif not entry.state.end_sequence:
                # readelf doesn't print the state after end_sequence
                # instructions. I think it's a bug but to be compatible
                # I don't print them too.
                if lp['version'] < 4:
                    print('%-35s  %11d  %18s' % (
                        lp['file_entry'][state.file - 1].name.decode('latin-1'),
                        state.line,
                        '0' if state.address == 0 else
                            hex(state.address)))
                else:
                    print('%-35s  %11d  %18s[%d]' % (
                        lp['file_entry'][state.file - 1].name.decode('latin-1'),
                        state.line,
                        '0' if state.address == 0 else
                            hex(state.address),
                        state.op_index))
            if entry.command == DW_LNS_copy:
                # Another readelf oddity...
                print()
            '''
            if entry.command in [DW_LNS_set_basic_block, DW_LNS_set_prologue_end, DW_LNS_set_isa]:
                print('=============================')
                print(repr(entry))
                print('=============================')
        #print(dir(lp))
        #line_entry_mapping(lp)

def line_entry_mapping(line_program):
    import collections
    filename_map = collections.defaultdict(int)
    lp_entries = line_program.get_entries()
    for lpe in lp_entries:
        if not lpe.state or lpe.state.file == 0:
            continue # TODO: instruction doesn't correspond to src
        filename = lpe_filename(line_program, lpe.state.file)
        filename_map[filename] += 1
    for filename, lpe_count in filename_map.items():
        print('%s -> %d entries' % (filename, lpe_count))

def lpe_filename(line_program, file_index):
    lp_header = line_program.header
    print(lp_header)
    file_entries = lp_header["file_entry"]
    file_entry = file_entries[file_index - 1]
    dir_index = file_entry["dir_index"]
    if dir_index == 0:
        return file_entry.name.decode()
    directory = lp_header["include_directory"][dir_index - 1]
    return os.path.join(directory, file_entry.name).decode()

    

#print(elf.get_section_by_name('.text').data())